**OBJETIVO**

O presente trabalho tem por objetivo criar uma base de dados no Databricks, adequar a estrutura e tratar os dados, a fim de analisar as questões propostas a seguir.

O Dataset utilizado contém dados que correlacionam usuários de diferentes tipos de drogas a diferentes tipos de personalidades (medidas pelo inventário psicológico NEO-FFI-R), além de variáveis essenciais como idade, nacionalidade e grau de escolaridade. Foi coletado desse repositório https://archive.ics.uci.edu/dataset/373/drug+consumption+quantified e possui licença CCA 4.0 International, estando livremente disponível para estudos.

Esse dataset já foi previamente utilizado por mim na diciplina de Machine Learning e, por isso, quis continuar a análise dele (dessa vez, de forma mais ampla). Por ser uma única tabela que já contém as informações que eu preciso e por precisar de diversas transformações como limpeza e mapeamentos categóricos, utilizarei o modelo de Datalake. Uma possível vantagem futura desse uso é a possibilidade de importar outros tipos de dados como dados clínicos e geográficos, em diferentes formatos.

Como se sabe, a política antidrogas é questão de saúde e seguraça pública, sendo de grande importância para a sociedade o melhor entendimento de fatores que podem acabar levando uma pessoa a usar drogas e até mesmo uma predição de condições favoráveis ao uso. Dessa forma, o sistema público poderia interferir de alguma maneira antes que a pessoa se viciasse. Acesso à educação, à saúde, acompanhamento psicológico e condições mínimas de vida são algumas opções que o poder público deve oferecer à sociedade pois contribuiriam para o afastamento e a redução do uso de drogas.

Neste trabalho, não consideraremos os diferentes tipos de personalidades pois são dados que requerem certo conhecimento de Psicologia para que possam ser analisados e eu não possuo esse conhecimento. Além disso, por questões de tempo e objetividade, decidi pelo recorte de 3 das drogas mais consumidas no mundo, que são a Maconha, a Cocaína e a Anfetamina. Porém mantive uma tabela tratada e com todas as drogas para possibilitar diferentes análises fora desse escopo proposto. Com isso, proponho as seguintes questões a serem respondidas:

1. Qual o perfil dos maiores usuários dessas drogas?
  1.1 Idade, Gênero e Escolaridade seguem algum padrão nesses casos?
2. Existe correlação entre consumo de Cannabis, Anfetaminas e Cocaína? (co-uso)
3. Como o poder púbico poderia atuar para reduzir a quantidade de usuários?




01 - Preparo de ambiente

Inicialmente, prepararei o ambiente criando o catálogo "mvp" e os schemas:

- staging: etapa em que os dados serão carregados na plataforma 
- bronze: etapa em que a tabela a ser trabalhada será criada e suas colunas catalogadas
- silver: etapa em que os dados serão transformados (ETL) para uma melhor análise
- gold: etapa em que a tabela final será criada e as análises serão executadas a fim de responder as questões propostas

In [0]:
%sql
DROP CATALOG IF EXISTS mvp CASCADE;
CREATE CATALOG mvp;
USE CATALOG mvp;
DROP SCHEMA IF EXISTS staging CASCADE;
CREATE SCHEMA staging;
DROP SCHEMA IF EXISTS bronze CASCADE;
CREATE SCHEMA bronze;
DROP SCHEMA IF EXISTS silver CASCADE;
CREATE SCHEMA silver;
DROP SCHEMA IF EXISTS gold CASCADE;
CREATE SCHEMA gold;

02 - Upload de dados

Aqui, o volume "drug_consumption" será criado para receber os dados em formato .csv. Conforme dito anteriormente, será utilizada uma única tabela contendo os dados necessários para a análise proposta.

O upload do arquivo .csv será feito de forma manual via interface gráfica do Databricks.


In [0]:
%sql
USE CATALOG mvp;
USE SCHEMA staging;
CREATE VOLUME drug_consumption

03 - Etapa Bronze

Nesta etapa, a tabela base e todo o catálogo serão criados. Esta tabela utiliza a vírgula como separador, não possui cabeçalho e os dados não estão em um formato legível (apenas numeros e codigos). Utilizando as informações disponíveis no site onde foi coletada, todas esses dados serão adaptados e transformados na etapa Silver.

In [0]:
spark.sql("USE CATALOG mvp")
spark.sql("USE SCHEMA bronze")

DataFrame[]

In [0]:
df = spark.read.option("header", False).option("sep", ",").csv("dbfs:/Volumes/mvp/staging/drug_consumption/drug_consumption.csv")
df.write.format("delta").mode("overwrite").saveAsTable("drug_consumption")
display(df.limit(10))

_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,_c10,_c11,_c12,_c13,_c14,_c15,_c16,_c17,_c18,_c19,_c20,_c21,_c22,_c23,_c24,_c25,_c26,_c27,_c28,_c29,_c30,_c31
1,49.788,48.246,-5.921,96.082,12.600,31.287,-57.545,-58.331,-91.699,-665,-21.712,-118.084,CL5,CL2,CL0,CL2,CL6,CL0,CL5,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL2,CL0,CL0
2,-7.854,-48.246,198.437,96.082,-31.685,-67.825,193.886,143.533,76.096,-14.277,-71.126,-21.575,CL5,CL2,CL2,CL0,CL6,CL4,CL6,CL3,CL0,CL4,CL0,CL2,CL0,CL2,CL3,CL0,CL4,CL0,CL0
3,49.788,-48.246,-5.921,96.082,-31.685,-46.725,80.523,-84.732,-162.090,-101.450,-137.983,40.148,CL6,CL0,CL0,CL0,CL6,CL3,CL4,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL1,CL0,CL0,CL0
4,-95.197,48.246,116.365,96.082,-31.685,-14.882,-80.615,-1.928,59.042,58.489,-137.983,-118.084,CL4,CL0,CL0,CL3,CL5,CL2,CL4,CL2,CL0,CL0,CL0,CL2,CL0,CL0,CL0,CL0,CL2,CL0,CL0
5,49.788,48.246,198.437,96.082,-31.685,73.545,-163.340,-45.174,-30.172,130.612,-21.712,-21.575,CL4,CL1,CL1,CL0,CL6,CL3,CL6,CL0,CL0,CL1,CL0,CL0,CL1,CL0,CL0,CL2,CL2,CL0,CL0
6,259.171,48.246,-122.751,24.923,-31.685,-67.825,-30.033,-155.521,203.972,163.088,-137.983,-154.858,CL2,CL0,CL0,CL0,CL6,CL0,CL4,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL6,CL0,CL0
7,109.449,-48.246,116.365,-57.009,-31.685,-46.725,-109.207,-45.174,-30.172,93.949,-21.712,7.987,CL6,CL0,CL0,CL0,CL6,CL1,CL5,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL6,CL0,CL0
8,49.788,-48.246,-173.790,96.082,-31.685,-132.828,193.886,-84.732,-30.172,163.088,19.268,-52.593,CL5,CL0,CL0,CL0,CL6,CL0,CL4,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0
9,49.788,48.246,-5.921,24.923,-31.685,62.967,257.309,-97.631,76.096,113.407,-137.983,-154.858,CL4,CL0,CL0,CL0,CL6,CL0,CL6,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL6,CL0,CL0
10,182.213,-48.246,116.365,96.082,-31.685,-24.649,332,-142.424,59.042,12.331,-137.983,-84.637,CL6,CL1,CL0,CL1,CL6,CL1,CL6,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL6,CL0,CL0


Os comentários a seguir foram copiados da fonte original, para garantir a confiabilidade das informações e dar o devido crédito aos criadores: https://archive.ics.uci.edu/dataset/373/drug+consumption+quantified

As colunas c6 a c12 não terão seus possíveis valores "traduzidos", pois não serão utilizadas no presente trabalho e necessitam de um conhecimento prévio de Psicologia para entender e analisar esses valores.

In [0]:
spark.sql("""
  COMMENT ON TABLE mvp.bronze.drug_consumption IS 
  'Database contains records for 1885 respondents. For each respondent 12 attributes are known: Personality measurements which include NEO-FFI-R (neuroticism, extraversion, openness to experience, agreeableness, and conscientiousness), BIS-11 (impulsivity), and ImpSS (sensation seeking), level of education, age, gender, country of residence and ethnicity. All input attributes are originally categorical and are quantified. After quantification values of all input features can be considered as real-valued. In addition, participants were questioned concerning their use of 18 legal and illegal drugs (alcohol, amphetamines, amyl nitrite, benzodiazepine, cannabis, chocolate, cocaine, caffeine, crack, ecstasy, heroin, ketamine, legal highs, LSD, methadone, mushrooms, nicotine and volatile substance abuse and one fictitious drug (Semeron) which was introduced to identify over-claimers. For each drug they have to select one of the answers: never used the drug, used it over a decade ago, or in the last decade, year, month, week, or day.
Database contains 18 classification problems. Each of independent label variables contains seven classes: "Never Used", "Used over a Decade Ago", "Used in Last Decade", "Used in Last Year", "Used in Last Month", "Used in Last Week", and "Used in Last Day".'
""")

DataFrame[]

In [0]:
spark.sql("""COMMENT ON COLUMN mvp.bronze.drug_consumption._c0 IS 'Number of record in original database. Cannot be related to participant. It can be used for reference only'""")
spark.sql("""COMMENT ON COLUMN mvp.bronze.drug_consumption._c1 IS 'Age of participant and has one of the values: -95.197: 18-24 / -7.854: 25-34 / 49.788: 35-44 / 109.449: 45-54 / 182.213: 55-64 / 259.171: 65+'""")
spark.sql("""COMMENT ON COLUMN mvp.bronze.drug_consumption._c2 IS 'Gender of participant: 48.246: Female / -48.246: Male'""")
spark.sql("""COMMENT ON COLUMN mvp.bronze.drug_consumption._c3 IS 'Level of education of participant and has one of the values: -243.591: Left school before 16 years / -173.790: Left school at 16 years / -143.719: Left school at 17 years / -122.751: Left school at 18 years / -61.113: Some college or university, no certificate or degree / -5.921: Professional certificate or diploma / 45.468: University degree / 116.365: Masters degree / 198.437: Doctorate degree'""")
spark.sql("""COMMENT ON COLUMN mvp.bronze.drug_consumption._c4 IS 'Country of current residence of participant and has one of the values: -9.765: Australia / 24.923: Canada / -46.841: New Zealand / -28.519: Other / 21.128: Republic of Ireland / 96.082: UK / -57.009: USA'""")
spark.sql("""COMMENT ON COLUMN mvp.bronze.drug_consumption._c5 IS 'Ethnicity of participant and has one of the values: -50.212: Asian / -110.702: Black / 190.725: Mixed-Black/Asian / 12.600: Mixed-White/Asian / -22.166: Mixed-White/Black / 11.440: Other / -31.685: White'""")
spark.sql("""COMMENT ON COLUMN mvp.bronze.drug_consumption._c6 IS 'Nscore is NEO-FFI-R Neuroticism.'""")
spark.sql("""COMMENT ON COLUMN mvp.bronze.drug_consumption._c7 IS 'Escore is NEO-FFI-R Extraversion.'""")
spark.sql("""COMMENT ON COLUMN mvp.bronze.drug_consumption._c8 IS 'Oscore is NEO-FFI-R Openness to experience.'""")
spark.sql("""COMMENT ON COLUMN mvp.bronze.drug_consumption._c9 IS 'Ascore is NEO-FFI-R Agreeableness.'""")
spark.sql("""COMMENT ON COLUMN mvp.bronze.drug_consumption._c10 IS 'Cscore is NEO-FFI-R Conscientiousness.'""")
spark.sql("""COMMENT ON COLUMN mvp.bronze.drug_consumption._c11 IS 'Impulsive is impulsiveness measured by BIS-11.'""")
spark.sql("""COMMENT ON COLUMN mvp.bronze.drug_consumption._c12 IS 'SS is sensation seeing measured by ImpSS.'""")
spark.sql("""COMMENT ON COLUMN mvp.bronze.drug_consumption._c13 IS 'Class of alcohol consumption. It is output attribute with following distribution of classes: CL0: Never Used / CL1: Used over a Decade Ago / CL2: Used in Last Decade / CL3: Used in Last Year / CL4: Used in Last Month / CL5: Used in Last Week / CL6: Used in Last Day'""")
spark.sql("""COMMENT ON COLUMN mvp.bronze.drug_consumption._c14 IS 'Class of amphetamines consumption. It is output attribute with following distribution of classes: CL0: Never Used / CL1: Used over a Decade Ago / CL2: Used in Last Decade / CL3: Used in Last Year / CL4: Used in Last Month / CL5: Used in Last Week / CL6: Used in Last Day'""")
spark.sql("""COMMENT ON COLUMN mvp.bronze.drug_consumption._c15 IS 'Class of amyl nitrite consumption. It is output attribute with following distribution of classes: CL0: Never Used / CL1: Used over a Decade Ago / CL2: Used in Last Decade / CL3: Used in Last Year / CL4: Used in Last Month / CL5: Used in Last Week / CL6: Used in Last Day'""")
spark.sql("""COMMENT ON COLUMN mvp.bronze.drug_consumption._c16 IS 'Class of benzodiazepine consumption. It is output attribute with following distribution of classes: CL0: Never Used / CL1: Used over a Decade Ago / CL2: Used in Last Decade / CL3: Used in Last Year / CL4: Used in Last Month / CL5: Used in Last Week / CL6: Used in Last Day'""")
spark.sql("""COMMENT ON COLUMN mvp.bronze.drug_consumption._c17 IS 'Class of caffeine consumption. It is output attribute with following distribution of classes: CL0: Never Used / CL1: Used over a Decade Ago / CL2: Used in Last Decade / CL3: Used in Last Year / CL4: Used in Last Month / CL5: Used in Last Week / CL6: Used in Last Day'""")
spark.sql("""COMMENT ON COLUMN mvp.bronze.drug_consumption._c18 IS 'Class of cannabis consumption. It is output attribute with following distribution of classes: CL0: Never Used / CL1: Used over a Decade Ago / CL2: Used in Last Decade / CL3: Used in Last Year / CL4: Used in Last Month / CL5: Used in Last Week / CL6: Used in Last Day'""")
spark.sql("""COMMENT ON COLUMN mvp.bronze.drug_consumption._c19 IS 'Class of chocolate consumption. It is output attribute with following distribution of classes: CL0: Never Used / CL1: Used over a Decade Ago / CL2: Used in Last Decade / CL3: Used in Last Year / CL4: Used in Last Month / CL5: Used in Last Week / CL6: Used in Last Day'""")
spark.sql("""COMMENT ON COLUMN mvp.bronze.drug_consumption._c20 IS 'Class of cocaine consumption. It is output attribute with following distribution of classes: CL0: Never Used / CL1: Used over a Decade Ago / CL2: Used in Last Decade / CL3: Used in Last Year / CL4: Used in Last Month / CL5: Used in Last Week / CL6: Used in Last Day'""")
spark.sql("""COMMENT ON COLUMN mvp.bronze.drug_consumption._c21 IS 'Class of crack consumption. It is output attribute with following distribution of classes: CL0: Never Used / CL1: Used over a Decade Ago / CL2: Used in Last Decade / CL3: Used in Last Year / CL4: Used in Last Month / CL5: Used in Last Week / CL6: Used in Last Day'""")
spark.sql("""COMMENT ON COLUMN mvp.bronze.drug_consumption._c22 IS 'Class of ecstasy consumption. It is output attribute with following distribution of classes: CL0: Never Used / CL1: Used over a Decade Ago / CL2: Used in Last Decade / CL3: Used in Last Year / CL4: Used in Last Month / CL5: Used in Last Week / CL6: Used in Last Day'""")
spark.sql("""COMMENT ON COLUMN mvp.bronze.drug_consumption._c23 IS 'Class of heroin consumption. It is output attribute with following distribution of classes: CL0: Never Used / CL1: Used over a Decade Ago / CL2: Used in Last Decade / CL3: Used in Last Year / CL4: Used in Last Month / CL5: Used in Last Week / CL6: Used in Last Day'""")
spark.sql("""COMMENT ON COLUMN mvp.bronze.drug_consumption._c24 IS 'Class of ketamine consumption. It is output attribute with following distribution of classes: CL0: Never Used / CL1: Used over a Decade Ago / CL2: Used in Last Decade / CL3: Used in Last Year / CL4: Used in Last Month / CL5: Used in Last Week / CL6: Used in Last Day'""")
spark.sql("""COMMENT ON COLUMN mvp.bronze.drug_consumption._c25 IS 'Class of legal highs consumption. It is output attribute with following distribution of classes: CL0: Never Used / CL1: Used over a Decade Ago / CL2: Used in Last Decade / CL3: Used in Last Year / CL4: Used in Last Month / CL5: Used in Last Week / CL6: Used in Last Day'""")
spark.sql("""COMMENT ON COLUMN mvp.bronze.drug_consumption._c26 IS 'Class of LSD consumption. It is output attribute with following distribution of classes: CL0: Never Used / CL1: Used over a Decade Ago / CL2: Used in Last Decade / CL3: Used in Last Year / CL4: Used in Last Month / CL5: Used in Last Week / CL6: Used in Last Day'""")
spark.sql("""COMMENT ON COLUMN mvp.bronze.drug_consumption._c27 IS 'Class of methadone consumption. It is output attribute with following distribution of classes: CL0: Never Used / CL1: Used over a Decade Ago / CL2: Used in Last Decade / CL3: Used in Last Year / CL4: Used in Last Month / CL5: Used in Last Week / CL6: Used in Last Day'""")
spark.sql("""COMMENT ON COLUMN mvp.bronze.drug_consumption._c28 IS 'Class of magic mushrooms consumption. It is output attribute with following distribution of classes: CL0: Never Used / CL1: Used over a Decade Ago / CL2: Used in Last Decade / CL3: Used in Last Year / CL4: Used in Last Month / CL5: Used in Last Week / CL6: Used in Last Day'""")
spark.sql("""COMMENT ON COLUMN mvp.bronze.drug_consumption._c29 IS 'Class of nicotine consumption. It is output attribute with following distribution of classes: CL0: Never Used / CL1: Used over a Decade Ago / CL2: Used in Last Decade / CL3: Used in Last Year / CL4: Used in Last Month / CL5: Used in Last Week / CL6: Used in Last Day'""")
spark.sql("""COMMENT ON COLUMN mvp.bronze.drug_consumption._c30 IS 'Class of fictitious drug Semeron consumption. It is output attribute with following distribution of classes: CL0: Never Used / CL1: Used over a Decade Ago / CL2: Used in Last Decade / CL3: Used in Last Year / CL4: Used in Last Month / CL5: Used in Last Week / CL6: Used in Last Day'""")
spark.sql("""COMMENT ON COLUMN mvp.bronze.drug_consumption._c31 IS 'Class of volatile substance abuse consumption. It is output attribute with following distribution of classes: CL0: Never Used / CL1: Used over a Decade Ago / CL2: Used in Last Decade / CL3: Used in Last Year / CL4: Used in Last Month / CL5: Used in Last Week / CL6: Used in Last Day'""")

DataFrame[]

04 - Etapa Silver

Aqui serão feitas todas as transformações e adequações necessárias para o tratamento dos dados.

A primeira tratativa será a nomeação correta das colunas, já que o csv não tinha cabeçalho e o sistema nomeou por default todas elas.

A segunda tratativa será o filtro de colunas que não serão usadas nesse trabalho e que não terão seus valores traduzidos para linguagem "real". São as colunas que indicam diferentes tipos de personalidades medidas conforme o inventário psicológico NEO-FFI-R e que precisam de certo conhecimento de outras áreas para melhor entendimento.

A terceira tratativa será traduzir os números para string, com valores que façam sentido para análise humana.

A quarta tratativa será o filtro das colunas de interesse para esse trabalho, que serão utilizadas nas análises. Com isso, farei a busca por valores nulos e fora do padrão.


In [0]:
spark.sql("USE CATALOG mvp")
spark.sql("USE SCHEMA silver")

DataFrame[]

Primeira tratativa - Rename de colunas (todas)

In [0]:
from pyspark.sql import DataFrame

#definindo a função rename para usar na sequencia
def rename_columns(df: DataFrame, new_names: dict) -> DataFrame:
    for old_name, new_name in new_names.items():
        df = df.withColumnRenamed(old_name, new_name)
    return df

#usando a função para renomear as colunas
df = spark.table("mvp.bronze.drug_consumption")

renames = {
    "_c0": "ID",
    "_c1": "Age",
    "_c2": "Gender",
    "_c3": "Education",
    "_c4": "Country",
    "_c5": "Ethnicity",
    "_c6": "Nscore",
    "_c7": "Escore",
    "_c8": "Oscore",
    "_c9": "Ascore",
    "_c10": "Cscore",
    "_c11": "Impulsive",
    "_c12": "SS",
    "_c13": "Alcohol",
    "_c14": "Amphet",
    "_c15": "Amyl",
    "_c16": "Benzos",
    "_c17": "Caff",
    "_c18": "Cannabis",
    "_c19": "Choc",
    "_c20": "Coke",
    "_c21": "Crack",
    "_c22": "Ecstasy",
    "_c23": "Heroin",
    "_c24": "Ketamine",
    "_c25": "LegalH",
    "_c26": "LSD",
    "_c27": "Methadone",
    "_c28": "Mushrooms",
    "_c29": "Nicotine",
    "_c30": "Semer",
    "_c31": "VSA"    
}

df_renamed = rename_columns(df, renames)

df_renamed.write.mode("overwrite").saveAsTable("drug_consumption")


In [0]:
%sql
SELECT *
FROM drug_consumption 
LIMIT 10

ID,Age,Gender,Education,Country,Ethnicity,Nscore,Escore,Oscore,Ascore,Cscore,Impulsive,SS,Alcohol,Amphet,Amyl,Benzos,Caff,Cannabis,Choc,Coke,Crack,Ecstasy,Heroin,Ketamine,LegalH,LSD,Methadone,Mushrooms,Nicotine,Semer,VSA
1,49.788,48.246,-5.921,96.082,12.600,31.287,-57.545,-58.331,-91.699,-665,-21.712,-118.084,CL5,CL2,CL0,CL2,CL6,CL0,CL5,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL2,CL0,CL0
2,-7.854,-48.246,198.437,96.082,-31.685,-67.825,193.886,143.533,76.096,-14.277,-71.126,-21.575,CL5,CL2,CL2,CL0,CL6,CL4,CL6,CL3,CL0,CL4,CL0,CL2,CL0,CL2,CL3,CL0,CL4,CL0,CL0
3,49.788,-48.246,-5.921,96.082,-31.685,-46.725,80.523,-84.732,-162.090,-101.450,-137.983,40.148,CL6,CL0,CL0,CL0,CL6,CL3,CL4,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL1,CL0,CL0,CL0
4,-95.197,48.246,116.365,96.082,-31.685,-14.882,-80.615,-1.928,59.042,58.489,-137.983,-118.084,CL4,CL0,CL0,CL3,CL5,CL2,CL4,CL2,CL0,CL0,CL0,CL2,CL0,CL0,CL0,CL0,CL2,CL0,CL0
5,49.788,48.246,198.437,96.082,-31.685,73.545,-163.340,-45.174,-30.172,130.612,-21.712,-21.575,CL4,CL1,CL1,CL0,CL6,CL3,CL6,CL0,CL0,CL1,CL0,CL0,CL1,CL0,CL0,CL2,CL2,CL0,CL0
6,259.171,48.246,-122.751,24.923,-31.685,-67.825,-30.033,-155.521,203.972,163.088,-137.983,-154.858,CL2,CL0,CL0,CL0,CL6,CL0,CL4,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL6,CL0,CL0
7,109.449,-48.246,116.365,-57.009,-31.685,-46.725,-109.207,-45.174,-30.172,93.949,-21.712,7.987,CL6,CL0,CL0,CL0,CL6,CL1,CL5,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL6,CL0,CL0
8,49.788,-48.246,-173.790,96.082,-31.685,-132.828,193.886,-84.732,-30.172,163.088,19.268,-52.593,CL5,CL0,CL0,CL0,CL6,CL0,CL4,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0
9,49.788,48.246,-5.921,24.923,-31.685,62.967,257.309,-97.631,76.096,113.407,-137.983,-154.858,CL4,CL0,CL0,CL0,CL6,CL0,CL6,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL6,CL0,CL0
10,182.213,-48.246,116.365,96.082,-31.685,-24.649,332,-142.424,59.042,12.331,-137.983,-84.637,CL6,CL1,CL0,CL1,CL6,CL1,CL6,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL6,CL0,CL0


Segunda tratativa - Drop de colunas que não serão usadas em nenhum momento

In [0]:
from pyspark.sql.functions import split, explode, trim, lower

df = spark.table("mvp.silver.drug_consumption")

df_expanded = df.drop("Nscore", "Escore", "Oscore", "Ascore", "Cscore", "Impulsive", "SS")

df_expanded.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("drug_consumption")

#df_expanded.write.mode("overwrite").saveAsTable("drug_consumption") - esse comando removeu somente os dados e não as colunas. por isso, tive que fazer o overwrite do schema também.

In [0]:
%sql
SELECT *
FROM drug_consumption 
LIMIT 10

ID,Age,Gender,Education,Country,Ethnicity,Alcohol,Amphet,Amyl,Benzos,Caff,Cannabis,Choc,Coke,Crack,Ecstasy,Heroin,Ketamine,LegalH,LSD,Methadone,Mushrooms,Nicotine,Semer,VSA
1,49.788,48.246,-5.921,96.082,12.600,CL5,CL2,CL0,CL2,CL6,CL0,CL5,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL2,CL0,CL0
2,-7.854,-48.246,198.437,96.082,-31.685,CL5,CL2,CL2,CL0,CL6,CL4,CL6,CL3,CL0,CL4,CL0,CL2,CL0,CL2,CL3,CL0,CL4,CL0,CL0
3,49.788,-48.246,-5.921,96.082,-31.685,CL6,CL0,CL0,CL0,CL6,CL3,CL4,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL1,CL0,CL0,CL0
4,-95.197,48.246,116.365,96.082,-31.685,CL4,CL0,CL0,CL3,CL5,CL2,CL4,CL2,CL0,CL0,CL0,CL2,CL0,CL0,CL0,CL0,CL2,CL0,CL0
5,49.788,48.246,198.437,96.082,-31.685,CL4,CL1,CL1,CL0,CL6,CL3,CL6,CL0,CL0,CL1,CL0,CL0,CL1,CL0,CL0,CL2,CL2,CL0,CL0
6,259.171,48.246,-122.751,24.923,-31.685,CL2,CL0,CL0,CL0,CL6,CL0,CL4,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL6,CL0,CL0
7,109.449,-48.246,116.365,-57.009,-31.685,CL6,CL0,CL0,CL0,CL6,CL1,CL5,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL6,CL0,CL0
8,49.788,-48.246,-173.790,96.082,-31.685,CL5,CL0,CL0,CL0,CL6,CL0,CL4,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0
9,49.788,48.246,-5.921,24.923,-31.685,CL4,CL0,CL0,CL0,CL6,CL0,CL6,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL6,CL0,CL0
10,182.213,-48.246,116.365,96.082,-31.685,CL6,CL1,CL0,CL1,CL6,CL1,CL6,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL6,CL0,CL0


Terceira tratativa - Tradução dos dados para linguagem de fácil compreensão

In [0]:
from itertools import chain
from pyspark.sql.functions import create_map, lit, col, trim
from pyspark.sql import DataFrame

#definindo uma função para aplicar em cada uma das colunas de forma mais rápida

def apply_mapping(df: DataFrame, col_name: str, mapping_dict: dict, new_col_name: str = None) -> DataFrame:
    """
    Substitui valores da coluna `col_name` de acordo com mapping_dict.
    Compara sempre como string + trim.
    """
    mapping_expr = create_map([lit(x) for x in chain(*mapping_dict.items())])
    target_col = new_col_name or col_name
    return df.withColumn(target_col, mapping_expr[trim(col(col_name).cast("string"))])


#mapeamento de valores
age_map = {
    "-95.197": "18-24",
    "-7.854": "25-34",
    "49.788": "35-44",
    "109.449": "45-54",
    "182.213": "55-64",
    "259.171": "65+"
}
gender_map = {
    "48.246": "Female",
    "-48.246": "Male"
}
education_map = {
    "-243.591": "Left school before 16",
    "-173.790": "Left school at 16",
    "-143.719": "Left school at 17",
    "-122.751": "Left school at 18",
    "-61.113": "Some college/university, no degree",
    "-5.921":  "Professional certificate/diploma",
    "45.468":  "University degree",
    "116.365": "Masters degree",
    "198.437": "Doctorate degree"
}
country_map = {
    "-9.765":  "Australia",
    "24.923":  "Canada",
    "-46.841": "New Zealand",
    "-28.519": "Other",
    "21.128":  "Republic of Ireland",
    "96.082":  "UK",
    "-57.009": "USA"
}
ethnicity_map = {
    "-50.212":  "Asian",
    "-110.702": "Black",
    "190.725":  "Mixed-Black/Asian",
    "12.600":   "Mixed-White/Asian",
    "-22.166":  "Mixed-White/Black",
    "11.440":   "Other",
    "-31.685":  "White"
}


#aplicando os mappings e salvando a nova tabela
df = spark.table("mvp.silver.drug_consumption")

df = apply_mapping(df, "Age", age_map)
df = apply_mapping(df, "Gender", gender_map)
df = apply_mapping(df, "Education", education_map)
df = apply_mapping(df, "Country", country_map)
df = apply_mapping(df, "Ethnicity", ethnicity_map)

df.write.mode("overwrite").saveAsTable("drug_consumption")


In [0]:
%sql
SELECT *
FROM drug_consumption
LIMIT 10

ID,Age,Gender,Education,Country,Ethnicity,Alcohol,Amphet,Amyl,Benzos,Caff,Cannabis,Choc,Coke,Crack,Ecstasy,Heroin,Ketamine,LegalH,LSD,Methadone,Mushrooms,Nicotine,Semer,VSA
1,35-44,Female,Professional certificate/diploma,UK,Mixed-White/Asian,CL5,CL2,CL0,CL2,CL6,CL0,CL5,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL2,CL0,CL0
2,25-34,Male,Doctorate degree,UK,White,CL5,CL2,CL2,CL0,CL6,CL4,CL6,CL3,CL0,CL4,CL0,CL2,CL0,CL2,CL3,CL0,CL4,CL0,CL0
3,35-44,Male,Professional certificate/diploma,UK,White,CL6,CL0,CL0,CL0,CL6,CL3,CL4,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL1,CL0,CL0,CL0
4,18-24,Female,Masters degree,UK,White,CL4,CL0,CL0,CL3,CL5,CL2,CL4,CL2,CL0,CL0,CL0,CL2,CL0,CL0,CL0,CL0,CL2,CL0,CL0
5,35-44,Female,Doctorate degree,UK,White,CL4,CL1,CL1,CL0,CL6,CL3,CL6,CL0,CL0,CL1,CL0,CL0,CL1,CL0,CL0,CL2,CL2,CL0,CL0
6,65+,Female,Left school at 18,Canada,White,CL2,CL0,CL0,CL0,CL6,CL0,CL4,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL6,CL0,CL0
7,45-54,Male,Masters degree,USA,White,CL6,CL0,CL0,CL0,CL6,CL1,CL5,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL6,CL0,CL0
8,35-44,Male,Left school at 16,UK,White,CL5,CL0,CL0,CL0,CL6,CL0,CL4,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0
9,35-44,Female,Professional certificate/diploma,Canada,White,CL4,CL0,CL0,CL0,CL6,CL0,CL6,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL6,CL0,CL0
10,55-64,Male,Masters degree,UK,White,CL6,CL1,CL0,CL1,CL6,CL1,CL6,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL6,CL0,CL0


As colunas de drogas utilizam os mesmos valores, então farei o mapping e aplicarei em todas elas.

In [0]:
cl_map = {
    "CL0": "Never used",
    "CL1": "Used over a decade ago",
    "CL2": "Used in last decade",
    "CL3": "Used in last year",
    "CL4": "Used in last month",
    "CL5": "Used in last week",
    "CL6": "Used in last day"
}

drug_columns = [
    "Alcohol", "Amphet", "Amyl", "Benzos", "Caff", "Cannabis", "Choc", "Coke", "Crack", "Ecstasy", "Heroin", "Ketamine", "LegalH", "LSD", "Methadone", "Mushrooms", "Nicotine", "Semer", "VSA"
]


from pyspark.sql.functions import trim, col

# lê a silver já com Age/Gender/Education/Country/Ethnicity tratados
df = spark.table("drug_consumption")

# garante que as colunas de droga não têm espaços extras
for c in drug_columns:
    df = df.withColumn(c, trim(col(c)))

# aplica o mapping CL0–CL6 em todas usando a função criada anteriormente
for c in drug_columns:
    df = apply_mapping(df, c, cl_map)

# salva tabela (pode ser a gold ou uma nova silver refinada)
df.write.mode("overwrite").saveAsTable("drug_consumption")


In [0]:
%sql
SELECT *
FROM drug_consumption
LIMIT 10

ID,Age,Gender,Education,Country,Ethnicity,Alcohol,Amphet,Amyl,Benzos,Caff,Cannabis,Choc,Coke,Crack,Ecstasy,Heroin,Ketamine,LegalH,LSD,Methadone,Mushrooms,Nicotine,Semer,VSA
1,35-44,Female,Professional certificate/diploma,UK,Mixed-White/Asian,Used in last week,Used in last decade,Never used,Used in last decade,Used in last day,Never used,Used in last week,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Used in last decade,Never used,Never used
2,25-34,Male,Doctorate degree,UK,White,Used in last week,Used in last decade,Used in last decade,Never used,Used in last day,Used in last month,Used in last day,Used in last year,Never used,Used in last month,Never used,Used in last decade,Never used,Used in last decade,Used in last year,Never used,Used in last month,Never used,Never used
3,35-44,Male,Professional certificate/diploma,UK,White,Used in last day,Never used,Never used,Never used,Used in last day,Used in last year,Used in last month,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Used over a decade ago,Never used,Never used,Never used
4,18-24,Female,Masters degree,UK,White,Used in last month,Never used,Never used,Used in last year,Used in last week,Used in last decade,Used in last month,Used in last decade,Never used,Never used,Never used,Used in last decade,Never used,Never used,Never used,Never used,Used in last decade,Never used,Never used
5,35-44,Female,Doctorate degree,UK,White,Used in last month,Used over a decade ago,Used over a decade ago,Never used,Used in last day,Used in last year,Used in last day,Never used,Never used,Used over a decade ago,Never used,Never used,Used over a decade ago,Never used,Never used,Used in last decade,Used in last decade,Never used,Never used
6,65+,Female,Left school at 18,Canada,White,Used in last decade,Never used,Never used,Never used,Used in last day,Never used,Used in last month,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Used in last day,Never used,Never used
7,45-54,Male,Masters degree,USA,White,Used in last day,Never used,Never used,Never used,Used in last day,Used over a decade ago,Used in last week,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Used in last day,Never used,Never used
8,35-44,Male,Left school at 16,UK,White,Used in last week,Never used,Never used,Never used,Used in last day,Never used,Used in last month,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used
9,35-44,Female,Professional certificate/diploma,Canada,White,Used in last month,Never used,Never used,Never used,Used in last day,Never used,Used in last day,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Used in last day,Never used,Never used
10,55-64,Male,Masters degree,UK,White,Used in last day,Used over a decade ago,Never used,Used over a decade ago,Used in last day,Used over a decade ago,Used in last day,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Used in last day,Never used,Never used


Durante essa etapa de mapping, percebi que os dados foram alterados quando fiz a conversão de .data (formato original) para .csv, os valores originais andaram duas casas decimais. 

O catálogo da etapa bronze foi simples de corrigir, só alterando os valores no código e rodando novamente. Porém o catálogo da silver continuou desatualizado e, por isso, farei essa etapa agora, já considerando os novos nomes das colunas e o filtro aplicado.

In [0]:
spark.sql("""COMMENT ON COLUMN mvp.silver.drug_consumption.ID IS 'Number of record in original database. Cannot be related to participant. It can be used for reference only'""")
spark.sql("""COMMENT ON COLUMN mvp.silver.drug_consumption.Age IS 'Age of participant and has one of the values: -95.197: 18-24 / -7.854: 25-34 / 49.788: 35-44 / 109.449: 45-54 / 182.213: 55-64 / 259.171: 65+'""")
spark.sql("""COMMENT ON COLUMN mvp.silver.drug_consumption.Gender IS 'Gender of participant: 48.246: Female / -48.246: Male'""")
spark.sql("""COMMENT ON COLUMN mvp.silver.drug_consumption.Education IS 'Level of education of participant and has one of the values: -243.591: Left school before 16 years / -173.790: Left school at 16 years / -143.719: Left school at 17 years / -122.751: Left school at 18 years / -61.113: Some college or university, no certificate or degree / -5.921: Professional certificate or diploma / 45.468: University degree / 116.365: Masters degree / 198.437: Doctorate degree'""")
spark.sql("""COMMENT ON COLUMN mvp.silver.drug_consumption.Country IS 'Country of current residence of participant and has one of the values: -9.765: Australia / 24.923: Canada / -46.841: New Zealand / -28.519: Other / 21.128: Republic of Ireland / 96.082: UK / -57.009: USA'""")
spark.sql("""COMMENT ON COLUMN mvp.silver.drug_consumption.Ethnicity IS 'Ethnicity of participant and has one of the values: -50.212: Asian / -110.702: Black / 190.725: Mixed-Black/Asian / 12.600: Mixed-White/Asian / -22.166: Mixed-White/Black / 11.440: Other / -31.685: White'""")

DataFrame[]

A quarta tratativa será a busca e, caso necessário, limpeza de valores NULL ou Unknown ou vazio. Ao final desse código, teremos uma tabela indicando a quantidade encontrada de cada um deles.

Como todas as transformações foram salvas na tabela padrão "drug_consumption", será essa a utilizada nessa busca. A tabela silver continuará como tabela base, somente filtrada e com os nomes de colunas corretos.

In [0]:
from pyspark.sql.functions import col, trim, sum as spark_sum
from pyspark.sql import Row

df = spark.table("drug_consumption")

#Monta as agregações (wide): null, Unknown, \N, blank
agg_exprs = []

for c in df.columns:
    null_expr       = spark_sum(col(c).isNull().cast("int")).alias(f"{c}_null_count")
    unknown_expr    = spark_sum((col(c) == "Unknown").cast("int")).alias(f"{c}_unknown_count")
    backslashN_expr = spark_sum((col(c) == "\\N").cast("int")).alias(f"{c}_backslashN_count")
    blank_expr      = spark_sum((trim(col(c)) == "").cast("int")).alias(f"{c}_blank_count")
    
    agg_exprs.extend([null_expr, unknown_expr, backslashN_expr, blank_expr])

df_checks_wide = df.agg(*agg_exprs)

#Converte o resultado wide (uma linha) em uma lista de linhas "long"
row = df_checks_wide.first()

data = []
for c in df.columns:
    null_count       = row[f"{c}_null_count"]
    unknown_count    = row[f"{c}_unknown_count"]
    backslashN_count = row[f"{c}_backslashN_count"]
    blank_count      = row[f"{c}_blank_count"]
    invalid_total    = null_count + unknown_count + backslashN_count + blank_count
    
    data.append(
        Row(
            column=c,
            null_count=null_count,
            unknown_count=unknown_count,
            backslashN_count=backslashN_count,
            blank_count=blank_count,
            invalid_total=invalid_total
        )
    )

#Cria o DataFrame longo com uma linha por coluna
df_checks_long = spark.createDataFrame(data)

#Ordena pela pior coluna (mais valores inválidos) e mostra tudo
df_checks_long.orderBy(col("invalid_total").desc()).show(1000, False)


+---------+----------+-------------+----------------+-----------+-------------+
|column   |null_count|unknown_count|backslashN_count|blank_count|invalid_total|
+---------+----------+-------------+----------------+-----------+-------------+
|ID       |0         |0            |0               |0          |0            |
|Age      |0         |0            |0               |0          |0            |
|Gender   |0         |0            |0               |0          |0            |
|Education|0         |0            |0               |0          |0            |
|Country  |0         |0            |0               |0          |0            |
|Ethnicity|0         |0            |0               |0          |0            |
|Alcohol  |0         |0            |0               |0          |0            |
|Amphet   |0         |0            |0               |0          |0            |
|Amyl     |0         |0            |0               |0          |0            |
|Benzos   |0         |0            |0   

Com isso, vemos que não existem dados inválidos nem nulos que pudessem atrapalhar as análises. Então seguiremos para a etapa Gold.

05 - Etapa Gold

Aqui aplicarei mais um filtro nas colunas de drogas, para deixar somente Cannabis, Cocaína e Anfetamina (conforme explicado no Objetivo do trabalho). Escolhi não criar esse filtro antes para que as outras drogas ainda estejam mais facilmente disponíveis para outros tipos de análise, além das propostas neste trabalho.

Também criarei uma nova coluna, chamada "Usuário", do tipo booleana para ajudar nas análises. Aqui, os valores "Never Used", "Used over a Decade Ago" e "Used in Last Decade" serão considerados como "Não Usuário". Os outros valores serão considerados como "Usuários".

Considerei a criação dessa coluna na etapa silver, antes do filtro final, mas eu teria que criar uma coluna de usuário para cada droga ou uma coluna genérica que considerasse o uso em pelo menos uma delas. Concluí que ambas as opções poderiam atrapalhar a etapa de análise e achei melhor criar agora somente.

In [0]:
#como a tabela final tratada na silver é a drug_consumption, criarei a gold como cópia dela
df_silver = spark.table("drug_consumption")
df_silver.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("mvp.gold.drug_consumption")


In [0]:
spark.sql("USE CATALOG mvp")
spark.sql("USE SCHEMA gold")

DataFrame[]

In [0]:
%sql
SELECT *
FROM mvp.gold.drug_consumption
LIMIT 10

ID,Age,Gender,Education,Country,Ethnicity,Alcohol,Amphet,Amyl,Benzos,Caff,Cannabis,Choc,Coke,Crack,Ecstasy,Heroin,Ketamine,LegalH,LSD,Methadone,Mushrooms,Nicotine,Semer,VSA
1,49.788,48.246,-5.921,96.082,12.600,CL5,CL2,CL0,CL2,CL6,CL0,CL5,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL2,CL0,CL0
2,-7.854,-48.246,198.437,96.082,-31.685,CL5,CL2,CL2,CL0,CL6,CL4,CL6,CL3,CL0,CL4,CL0,CL2,CL0,CL2,CL3,CL0,CL4,CL0,CL0
3,49.788,-48.246,-5.921,96.082,-31.685,CL6,CL0,CL0,CL0,CL6,CL3,CL4,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL1,CL0,CL0,CL0
4,-95.197,48.246,116.365,96.082,-31.685,CL4,CL0,CL0,CL3,CL5,CL2,CL4,CL2,CL0,CL0,CL0,CL2,CL0,CL0,CL0,CL0,CL2,CL0,CL0
5,49.788,48.246,198.437,96.082,-31.685,CL4,CL1,CL1,CL0,CL6,CL3,CL6,CL0,CL0,CL1,CL0,CL0,CL1,CL0,CL0,CL2,CL2,CL0,CL0
6,259.171,48.246,-122.751,24.923,-31.685,CL2,CL0,CL0,CL0,CL6,CL0,CL4,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL6,CL0,CL0
7,109.449,-48.246,116.365,-57.009,-31.685,CL6,CL0,CL0,CL0,CL6,CL1,CL5,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL6,CL0,CL0
8,49.788,-48.246,-173.790,96.082,-31.685,CL5,CL0,CL0,CL0,CL6,CL0,CL4,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0
9,49.788,48.246,-5.921,24.923,-31.685,CL4,CL0,CL0,CL0,CL6,CL0,CL6,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL6,CL0,CL0
10,182.213,-48.246,116.365,96.082,-31.685,CL6,CL1,CL0,CL1,CL6,CL1,CL6,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL0,CL6,CL0,CL0


Nessa etapa, percebi que algo de errado aconteceu com a tabela refinada. Descobri que ela acabou sendo salva no catalogo chamado "workspace" e não no "mvp". No catálogo "mvp", no schema "silver", a drug_consumption está com os dados base após a primeira tratativa, que foi o nome das colunas.

Então, como eu quero preservar essa tabela base da silver, vou copiar a refinada criando uma nova tabela na silver chamada "drug_consumption_refined" para que essa seja a utilizada na etapa gold.

In [0]:
df_refined = spark.table("workspace.default.drug_consumption")
df_refined.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("mvp.silver.drug_consumption_refined")


In [0]:
%sql
SELECT *
FROM mvp.silver.drug_consumption_refined
LIMIT 10

ID,Age,Gender,Education,Country,Ethnicity,Alcohol,Amphet,Amyl,Benzos,Caff,Cannabis,Choc,Coke,Crack,Ecstasy,Heroin,Ketamine,LegalH,LSD,Methadone,Mushrooms,Nicotine,Semer,VSA
1,35-44,Female,Professional certificate/diploma,UK,Mixed-White/Asian,Used in last week,Used in last decade,Never used,Used in last decade,Used in last day,Never used,Used in last week,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Used in last decade,Never used,Never used
2,25-34,Male,Doctorate degree,UK,White,Used in last week,Used in last decade,Used in last decade,Never used,Used in last day,Used in last month,Used in last day,Used in last year,Never used,Used in last month,Never used,Used in last decade,Never used,Used in last decade,Used in last year,Never used,Used in last month,Never used,Never used
3,35-44,Male,Professional certificate/diploma,UK,White,Used in last day,Never used,Never used,Never used,Used in last day,Used in last year,Used in last month,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Used over a decade ago,Never used,Never used,Never used
4,18-24,Female,Masters degree,UK,White,Used in last month,Never used,Never used,Used in last year,Used in last week,Used in last decade,Used in last month,Used in last decade,Never used,Never used,Never used,Used in last decade,Never used,Never used,Never used,Never used,Used in last decade,Never used,Never used
5,35-44,Female,Doctorate degree,UK,White,Used in last month,Used over a decade ago,Used over a decade ago,Never used,Used in last day,Used in last year,Used in last day,Never used,Never used,Used over a decade ago,Never used,Never used,Used over a decade ago,Never used,Never used,Used in last decade,Used in last decade,Never used,Never used
6,65+,Female,Left school at 18,Canada,White,Used in last decade,Never used,Never used,Never used,Used in last day,Never used,Used in last month,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Used in last day,Never used,Never used
7,45-54,Male,Masters degree,USA,White,Used in last day,Never used,Never used,Never used,Used in last day,Used over a decade ago,Used in last week,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Used in last day,Never used,Never used
8,35-44,Male,Left school at 16,UK,White,Used in last week,Never used,Never used,Never used,Used in last day,Never used,Used in last month,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used
9,35-44,Female,Professional certificate/diploma,Canada,White,Used in last month,Never used,Never used,Never used,Used in last day,Never used,Used in last day,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Used in last day,Never used,Never used
10,55-64,Male,Masters degree,UK,White,Used in last day,Used over a decade ago,Never used,Used over a decade ago,Used in last day,Used over a decade ago,Used in last day,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Used in last day,Never used,Never used


Agora temos a tabela refinada na silver. Vou criar a gold a partir dessa.

In [0]:
df_silver_refined = spark.table("mvp.silver.drug_consumption_refined")

df_silver_refined.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("mvp.gold.drug_consumption")


In [0]:
%sql
SELECT *
FROM mvp.gold.drug_consumption
LIMIT 10

ID,Age,Gender,Education,Country,Ethnicity,Alcohol,Amphet,Amyl,Benzos,Caff,Cannabis,Choc,Coke,Crack,Ecstasy,Heroin,Ketamine,LegalH,LSD,Methadone,Mushrooms,Nicotine,Semer,VSA
1,35-44,Female,Professional certificate/diploma,UK,Mixed-White/Asian,Used in last week,Used in last decade,Never used,Used in last decade,Used in last day,Never used,Used in last week,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Used in last decade,Never used,Never used
2,25-34,Male,Doctorate degree,UK,White,Used in last week,Used in last decade,Used in last decade,Never used,Used in last day,Used in last month,Used in last day,Used in last year,Never used,Used in last month,Never used,Used in last decade,Never used,Used in last decade,Used in last year,Never used,Used in last month,Never used,Never used
3,35-44,Male,Professional certificate/diploma,UK,White,Used in last day,Never used,Never used,Never used,Used in last day,Used in last year,Used in last month,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Used over a decade ago,Never used,Never used,Never used
4,18-24,Female,Masters degree,UK,White,Used in last month,Never used,Never used,Used in last year,Used in last week,Used in last decade,Used in last month,Used in last decade,Never used,Never used,Never used,Used in last decade,Never used,Never used,Never used,Never used,Used in last decade,Never used,Never used
5,35-44,Female,Doctorate degree,UK,White,Used in last month,Used over a decade ago,Used over a decade ago,Never used,Used in last day,Used in last year,Used in last day,Never used,Never used,Used over a decade ago,Never used,Never used,Used over a decade ago,Never used,Never used,Used in last decade,Used in last decade,Never used,Never used
6,65+,Female,Left school at 18,Canada,White,Used in last decade,Never used,Never used,Never used,Used in last day,Never used,Used in last month,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Used in last day,Never used,Never used
7,45-54,Male,Masters degree,USA,White,Used in last day,Never used,Never used,Never used,Used in last day,Used over a decade ago,Used in last week,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Used in last day,Never used,Never used
8,35-44,Male,Left school at 16,UK,White,Used in last week,Never used,Never used,Never used,Used in last day,Never used,Used in last month,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used
9,35-44,Female,Professional certificate/diploma,Canada,White,Used in last month,Never used,Never used,Never used,Used in last day,Never used,Used in last day,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Used in last day,Never used,Never used
10,55-64,Male,Masters degree,UK,White,Used in last day,Used over a decade ago,Never used,Used over a decade ago,Used in last day,Used over a decade ago,Used in last day,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Never used,Used in last day,Never used,Never used


Agora sim posso seguir com as últimas tratativas na gold.

Aplicando o filtro para termos somente as colunas de drogas Cannabis, Cocaína e Anfetamina.

In [0]:
df = spark.table("mvp.gold.drug_consumption")

df_filtered = df.select(
    "Age", "Gender", "Education", "Country", "Ethnicity", "Cannabis", "Coke", "Amphet"
)

df_filtered.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("mvp.gold.drug_consumption_analytics")


In [0]:
%sql
SELECT *
FROM mvp.gold.drug_consumption_analytics
LIMIT 10

Age,Gender,Education,Country,Ethnicity,Cannabis,Coke,Amphet
35-44,Female,Professional certificate/diploma,UK,Mixed-White/Asian,Never used,Never used,Used in last decade
25-34,Male,Doctorate degree,UK,White,Used in last month,Used in last year,Used in last decade
35-44,Male,Professional certificate/diploma,UK,White,Used in last year,Never used,Never used
18-24,Female,Masters degree,UK,White,Used in last decade,Used in last decade,Never used
35-44,Female,Doctorate degree,UK,White,Used in last year,Never used,Used over a decade ago
65+,Female,Left school at 18,Canada,White,Never used,Never used,Never used
45-54,Male,Masters degree,USA,White,Used over a decade ago,Never used,Never used
35-44,Male,Left school at 16,UK,White,Never used,Never used,Never used
35-44,Female,Professional certificate/diploma,Canada,White,Never used,Never used,Never used
55-64,Male,Masters degree,UK,White,Used over a decade ago,Never used,Used over a decade ago


Agora vou criar uma nova coluna, chamada "Usuário", do tipo booleana para ajudar nas análises. Aqui, os valores "Never Used", "Used over a Decade Ago" e "Used in Last Decade" serão considerados como "Não Usuário" (false). Os outros valores serão considerados como "Usuários" (true).

Será uma coluna única que considerará como usuário alguém que usou pelo menos uma das 3 drogas listadas nessa etapa.

In [0]:
from pyspark.sql.functions import col

df = spark.table("mvp.gold.drug_consumption_analytics")

# valores que representam NÃO USUÁRIO
non_user_values = [
    "Never used",
    "Used over a decade ago",
    "Used in last decade"
]

# criar coluna geral: usuario se usar qualquer uma das 3 drogas
df = df.withColumn(
    "Usuario",
    ~col("Cannabis").isin(non_user_values) |
    ~col("Coke").isin(non_user_values) |
    ~col("Amphet").isin(non_user_values)
)


df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("mvp.gold.drug_consumption_analytics")


In [0]:
%sql
SELECT *
FROM mvp.gold.drug_consumption_analytics
LIMIT 10

Age,Gender,Education,Country,Ethnicity,Cannabis,Coke,Amphet,Usuario
35-44,Female,Professional certificate/diploma,UK,Mixed-White/Asian,Never used,Never used,Used in last decade,false
25-34,Male,Doctorate degree,UK,White,Used in last month,Used in last year,Used in last decade,true
35-44,Male,Professional certificate/diploma,UK,White,Used in last year,Never used,Never used,true
18-24,Female,Masters degree,UK,White,Used in last decade,Used in last decade,Never used,false
35-44,Female,Doctorate degree,UK,White,Used in last year,Never used,Used over a decade ago,true
65+,Female,Left school at 18,Canada,White,Never used,Never used,Never used,false
45-54,Male,Masters degree,USA,White,Used over a decade ago,Never used,Never used,false
35-44,Male,Left school at 16,UK,White,Never used,Never used,Never used,false
35-44,Female,Professional certificate/diploma,Canada,White,Never used,Never used,Never used,false
55-64,Male,Masters degree,UK,White,Used over a decade ago,Never used,Used over a decade ago,false


Pronto, agora temos a tabela que será utilizada como referência para responder as perguntas propostas no objetivo do trabalho.

ANÁLISES

Agora, iniciaremos as análises desse dataset, tendo como base a tabela mvp.gold.drug_consumption_analytics, para responder as seguintes perguntas:

1. Qual o perfil dos maiores usuários dessas drogas?

In [0]:
spark.sql("USE CATALOG mvp")
spark.sql("USE SCHEMA gold")

DataFrame[]

Para facilitar o cruzamento de dados em análise e para não "engessar" a atualização de novos inputs, ao invés de criar uma tabela-fato preferi criar uma view-fato, a qual será a minha base de análise já considerando os usuários e com todos os atributos que preciso. A partir dela, criarei as views dimensionais para as análises específicas e os cruzamentos de dados.

In [0]:
%sql
CREATE OR REPLACE VIEW mvp.gold.v_drug_usage_fact AS
SELECT *,
  -- usuário de cada droga
  CASE
    WHEN Cannabis IN ('Used in last year','Used in last month','Used in last week','Used in last day')
    THEN TRUE ELSE FALSE
  END AS Usuario_Cannabis,
  CASE
    WHEN Coke IN ('Used in last year','Used in last month','Used in last week','Used in last day')
    THEN TRUE ELSE FALSE
  END AS Usuario_Coke,
  CASE
    WHEN Amphet IN ('Used in last year','Used in last month','Used in last week','Used in last day')
    THEN TRUE ELSE FALSE
  END AS Usuario_Amphet
FROM mvp.gold.drug_consumption_analytics;



In [0]:
%sql
select * from mvp.gold.v_drug_usage_fact;

Age,Gender,Education,Country,Ethnicity,Cannabis,Coke,Amphet,Usuario,Usuario_Cannabis,Usuario_Coke,Usuario_Amphet
35-44,Female,Professional certificate/diploma,UK,Mixed-White/Asian,Never used,Never used,Used in last decade,false,false,false,false
25-34,Male,Doctorate degree,UK,White,Used in last month,Used in last year,Used in last decade,true,true,true,false
35-44,Male,Professional certificate/diploma,UK,White,Used in last year,Never used,Never used,true,true,false,false
18-24,Female,Masters degree,UK,White,Used in last decade,Used in last decade,Never used,false,false,false,false
35-44,Female,Doctorate degree,UK,White,Used in last year,Never used,Used over a decade ago,true,true,false,false
65+,Female,Left school at 18,Canada,White,Never used,Never used,Never used,false,false,false,false
45-54,Male,Masters degree,USA,White,Used over a decade ago,Never used,Never used,false,false,false,false
35-44,Male,Left school at 16,UK,White,Never used,Never used,Never used,false,false,false,false
35-44,Female,Professional certificate/diploma,Canada,White,Never used,Never used,Never used,false,false,false,false
55-64,Male,Masters degree,UK,White,Used over a decade ago,Never used,Used over a decade ago,false,false,false,false


Agora criarei as views dimensionais para cada classe que quero analisar: idade, gênero, escolaridade e país.

In [0]:
%sql
CREATE OR REPLACE VIEW mvp.gold.v_usage_by_age AS
SELECT
  Age,
  COUNT(*) AS total_pessoas,
  SUM(CASE WHEN Usuario THEN 1 ELSE 0 END) AS usuarios_qualquer_droga,
  ROUND(100.0 * SUM(CASE WHEN Usuario THEN 1 ELSE 0 END) / COUNT(*), 1) AS pct_usuarios_qualquer_droga,
  SUM(CASE WHEN Usuario_Cannabis THEN 1 ELSE 0 END) AS usuarios_cannabis,
  ROUND(100.0 * SUM(CASE WHEN Usuario_Cannabis THEN 1 ELSE 0 END) / COUNT(*), 1) AS pct_usuarios_cannabis,
  SUM(CASE WHEN Usuario_Coke THEN 1 ELSE 0 END) AS usuarios_coke,
  ROUND(100.0 * SUM(CASE WHEN Usuario_Coke THEN 1 ELSE 0 END) / COUNT(*), 1) AS pct_usuarios_coke,
  SUM(CASE WHEN Usuario_Amphet THEN 1 ELSE 0 END) AS usuarios_amphet,
  ROUND(100.0 * SUM(CASE WHEN Usuario_Amphet THEN 1 ELSE 0 END) / COUNT(*), 1) AS pct_usuarios_amphet
FROM mvp.gold.v_drug_usage_fact
GROUP BY Age
ORDER BY Age;


In [0]:
%sql
select * from mvp.gold.v_usage_by_age;

Age,total_pessoas,usuarios_qualquer_droga,pct_usuarios_qualquer_droga,usuarios_cannabis,pct_usuarios_cannabis,usuarios_coke,pct_usuarios_coke,usuarios_amphet,pct_usuarios_amphet
18-24,643,550,85.5,536,83.4,226,35.1,259,40.3
25-34,481,260,54.1,238,49.5,122,25.4,107,22.2
35-44,356,136,38.2,123,34.6,45,12.6,36,10.1
45-54,294,85,28.9,75,25.5,20,6.8,31,10.5
55-64,93,28,30.1,26,28.0,4,4.3,3,3.2
65+,18,1,5.6,1,5.6,0,0.0,0,0.0


Aqui temos a porcentagem de usuários recentes, que usou minimamente no último ano, para cada droga e organizado por idade. Vemos que a droga mais utilizada dentre as 3 é a maconha em todos as faixas etárias. Já a anfetamina supera a cocaína em 2 faixas: 18-24 (os mais jovens) e 45-54 (os de meia idade).

Temos também a quantidade de usuários de pelo menos uma das 3 drogas (usuario_qualquer_droga). Quase todos os jovens podem ser considerados usuários. Podemos observar que a quantidade de usuários vai diminuindo conforme a faixa etária vai aumentando.

In [0]:
%sql
CREATE OR REPLACE VIEW mvp.gold.v_usage_by_gender AS
SELECT
  Gender,
  COUNT(*) AS total_pessoas,
  SUM(CASE WHEN Usuario THEN 1 ELSE 0 END) AS usuarios_qualquer_droga,
  ROUND(100.0 * SUM(CASE WHEN Usuario THEN 1 ELSE 0 END) / COUNT(*), 1) AS pct_usuarios_qualquer_droga,
  SUM(CASE WHEN Usuario_Cannabis THEN 1 ELSE 0 END) AS usuarios_cannabis,
  ROUND(100.0 * SUM(CASE WHEN Usuario_Cannabis THEN 1 ELSE 0 END) / COUNT(*), 1) AS pct_usuarios_cannabis,
  SUM(CASE WHEN Usuario_Coke THEN 1 ELSE 0 END) AS usuarios_coke,
  ROUND(100.0 * SUM(CASE WHEN Usuario_Coke THEN 1 ELSE 0 END) / COUNT(*), 1) AS pct_usuarios_coke,
  SUM(CASE WHEN Usuario_Amphet THEN 1 ELSE 0 END) AS usuarios_amphet,
  ROUND(100.0 * SUM(CASE WHEN Usuario_Amphet THEN 1 ELSE 0 END) / COUNT(*), 1) AS pct_usuarios_amphet
FROM mvp.gold.v_drug_usage_fact
GROUP BY Gender
ORDER BY Gender;


In [0]:
%sql
select * from mvp.gold.v_usage_by_gender;

Gender,total_pessoas,usuarios_qualquer_droga,pct_usuarios_qualquer_droga,usuarios_cannabis,pct_usuarios_cannabis,usuarios_coke,pct_usuarios_coke,usuarios_amphet,pct_usuarios_amphet
Female,942,395,41.9,364,38.6,146,15.5,137,14.5
Male,943,665,70.5,635,67.3,271,28.7,299,31.7


Aqui temos uma visualização por gênero, onde fica claro que a quantidade de homens usuário é quase o dobro da quantidade de mulheres usuárias, tanto quando generalizamos as drogas quanto em cada uma delas.

A maconha segue sendo a mais consumida, mas nas outras duas temos uma inversão. Mulheres consomem mais cocaína e homens consomem mais anfetamina.

In [0]:
%sql
CREATE OR REPLACE VIEW mvp.gold.v_usage_by_education AS
SELECT
  Education,
  COUNT(*) AS total_pessoas,
  SUM(CASE WHEN Usuario THEN 1 ELSE 0 END) AS usuarios_qualquer_droga,
  ROUND(100.0 * SUM(CASE WHEN Usuario THEN 1 ELSE 0 END) / COUNT(*), 1) AS pct_usuarios_qualquer_droga,
  SUM(CASE WHEN Usuario_Cannabis THEN 1 ELSE 0 END) AS usuarios_cannabis,
  ROUND(100.0 * SUM(CASE WHEN Usuario_Cannabis THEN 1 ELSE 0 END) / COUNT(*), 1) AS pct_usuarios_cannabis,
  SUM(CASE WHEN Usuario_Coke THEN 1 ELSE 0 END) AS usuarios_coke,
  ROUND(100.0 * SUM(CASE WHEN Usuario_Coke THEN 1 ELSE 0 END) / COUNT(*), 1) AS pct_usuarios_coke,
  SUM(CASE WHEN Usuario_Amphet THEN 1 ELSE 0 END) AS usuarios_amphet,
  ROUND(100.0 * SUM(CASE WHEN Usuario_Amphet THEN 1 ELSE 0 END) / COUNT(*), 1) AS pct_usuarios_amphet
FROM mvp.gold.v_drug_usage_fact
GROUP BY Education
ORDER BY Education;


In [0]:
%sql
select * from mvp.gold.v_usage_by_education;

Education,total_pessoas,usuarios_qualquer_droga,pct_usuarios_qualquer_droga,usuarios_cannabis,pct_usuarios_cannabis,usuarios_coke,pct_usuarios_coke,usuarios_amphet,pct_usuarios_amphet
Doctorate degree,89,36,40.4,31,34.8,15,16.9,11,12.4
Left school at 16,99,48,48.5,44,44.4,15,15.2,15,15.2
Left school at 17,30,21,70.0,20,66.7,9,30.0,9,30.0
Left school at 18,100,70,70.0,69,69.0,24,24.0,36,36.0
Left school before 16,28,19,67.9,18,64.3,5,17.9,6,21.4
Masters degree,283,96,33.9,87,30.7,40,14.1,30,10.6
Professional certificate/diploma,270,126,46.7,118,43.7,53,19.6,42,15.6
"Some college/university, no degree",506,425,84.0,408,80.6,169,33.4,207,40.9
University degree,480,219,45.6,204,42.5,87,18.1,80,16.7


Agora em relação ao grau de escolaridade, vemos que a maior parte dos usuários está no grupo de pessoas com certo grau de escolaridade, que são as pessoas com diploma ou que já frequentaram universidades/faculdades sem concluí-las. Curiosamente, os menos números estão nos extremos, entre pessoas com baixo grau de escolaridade e pessoas com os mais altos graus de escolaridade.

Com essas views criadas, podemos fazer algumas consultas para concluir o questionamento sobre o perfil dos maiores usuários e se existe um padrão nas características.

Primeiro, veremos as 10 combinações mais frequentes de usuários.

In [0]:
%sql
SELECT Age, Gender, Education,
  COUNT(*) AS total_usuarios,
  ROUND(
    100.0 * COUNT(*) /
    (SELECT COUNT(*) FROM mvp.gold.v_drug_usage_fact WHERE Usuario = TRUE),
    1
  ) AS pct_dos_usuarios
FROM mvp.gold.v_drug_usage_fact
WHERE Usuario = TRUE
GROUP BY Age, Gender, Education
ORDER BY total_usuarios DESC
LIMIT 10;


Age,Gender,Education,total_usuarios,pct_dos_usuarios
18-24,Male,"Some college/university, no degree",227,21.4
18-24,Female,"Some college/university, no degree",84,7.9
25-34,Male,"Some college/university, no degree",53,5.0
18-24,Male,University degree,47,4.4
25-34,Female,University degree,44,4.2
18-24,Male,Left school at 18,42,4.0
18-24,Female,University degree,40,3.8
25-34,Female,Masters degree,36,3.4
18-24,Male,Professional certificate/diploma,34,3.2
25-34,Male,University degree,27,2.5


Confirmando o que vimos nas views, a maior quantidade de usuários é formada por homens jovens que não concluíram o ensino superior (aqui fica a dúvida sobre esse dataset, se essa categoria de edução pode significar que ainda estão cursando). Essa quantidade é quase o triplo do segundo grupo de mesmas características, porém formado por mulheres.

Podemos perceber que as características de gênero e escolaridade variam bastante, enquanto que a faixa etária permanece nos jovens de 18 a 34 anos, mostrando que há um ponto em comum entre esses usuários.

Por fim, faremos uma consulta para verificar os perfis mais frequentes considerando cada uma das drogas. Assim, veremos quais drogas os maiores usuários utilizam.

In [0]:
%sql
WITH exploded AS (
  SELECT Age, Gender, Education,
    -- "desempilha" as 3 drogas em linhas:
    STACK(
      3,
      'Cannabis', Usuario_Cannabis,
      'Coke',     Usuario_Coke,
      'Amphet',   Usuario_Amphet
    ) AS (droga, is_user)
  FROM mvp.gold.v_drug_usage_fact
),
agg AS (
  SELECT droga, Age, Gender, Education,
    COUNT(*) AS total_usuarios_droga,
    -- % dentro do universo daquela droga
    ROUND(
      100.0 * COUNT(*) /
      SUM(COUNT(*)) OVER (PARTITION BY droga),
      1
    ) AS pct_usuarios_droga
  FROM exploded
  WHERE is_user = TRUE
  GROUP BY droga, Age, Gender, Education
),
ranked AS (
  SELECT droga, Age, Gender, Education, total_usuarios_droga, pct_usuarios_droga,
    ROW_NUMBER() OVER (
      PARTITION BY droga
      ORDER BY total_usuarios_droga DESC
    ) AS rn
  FROM agg
)
SELECT droga, Age, Gender, Education, total_usuarios_droga, pct_usuarios_droga
FROM ranked
WHERE rn <= 5
ORDER BY total_usuarios_droga DESC;


droga,Age,Gender,Education,total_usuarios_droga,pct_usuarios_droga
Cannabis,18-24,Male,"Some college/university, no degree",223,22.3
Amphet,18-24,Male,"Some college/university, no degree",124,28.4
Coke,18-24,Male,"Some college/university, no degree",94,22.5
Cannabis,18-24,Female,"Some college/university, no degree",81,8.1
Cannabis,18-24,Male,University degree,47,4.7
Cannabis,25-34,Male,"Some college/university, no degree",47,4.7
Cannabis,18-24,Male,Left school at 18,42,4.2
Amphet,18-24,Female,"Some college/university, no degree",33,7.6
Coke,18-24,Female,"Some college/university, no degree",30,7.2
Amphet,25-34,Male,"Some college/university, no degree",28,6.4


Com isso, temos a confirmação de um perfil predominante para todas as 3 drogas: jovens homens sem formação superior concluída (ou em andamento). Podemos analisar esse perfil como sendo homens no início da vida adulta, querendo viver novas experiências trazidas pela liberdade da idade. Estando com o ensino superior em andamento, estão vivenciando o ambiente das faculdades e universidades, onde acontecem muitas festas. A presença da maconha no topo da lista é facilmente justificável pela facilidade que se tem de obter a droga e até mesmo produzir. Já a anfetamina me remete a ambientes de festas, típicos de universidades e eventos jovens.

Agora vamos para a segunda pergunta:

2. Existe correlação entre consumo de Cannabis, Anfetaminas e Cocaína? (co-uso)

Primeiro, vamos ver quantas pessoas usam mais de uma droga ao mesmo tempo e qual o perfil dos co-usuários. Combinarei as colunas de usuários independente do perfil, inicialmente, para termos as quantidades independentes e combinadas. Na segunda consulta, voltaremos com as características e olharemos para o total de usuários independente da droga.

In [0]:
%sql
SELECT
  SUM(CASE WHEN Usuario_Cannabis THEN 1 ELSE 0 END) AS cannabis_users,
  SUM(CASE WHEN Usuario_Coke THEN 1 ELSE 0 END) AS coke_users,
  SUM(CASE WHEN Usuario_Amphet THEN 1 ELSE 0 END) AS amphet_users,

  -- co-uso
  SUM(CASE WHEN Usuario_Cannabis AND Usuario_Coke THEN 1 ELSE 0 END) AS cannabis_coke_co_use,
  SUM(CASE WHEN Usuario_Cannabis AND Usuario_Amphet THEN 1 ELSE 0 END) AS cannabis_amphet_co_use,
  SUM(CASE WHEN Usuario_Coke AND Usuario_Amphet THEN 1 ELSE 0 END) AS coke_amphet_co_use,

  -- co-uso das 3
  SUM(CASE WHEN Usuario_Cannabis AND Usuario_Coke AND Usuario_Amphet THEN 1 ELSE 0 END) AS all_three
FROM mvp.gold.v_drug_usage_fact;


cannabis_users,coke_users,amphet_users,cannabis_coke_co_use,cannabis_amphet_co_use,coke_amphet_co_use,all_three
999,417,436,374,398,246,226


In [0]:
%sql
SELECT Age, Gender, Education,
  COUNT(*) AS total_co_users
FROM mvp.gold.v_drug_usage_fact
WHERE 
    CAST(Usuario_Cannabis AS INT) + CAST(Usuario_Coke AS INT) + CAST(Usuario_Amphet AS INT) >= 2
GROUP BY Age, Gender, Education
ORDER BY total_co_users DESC
LIMIT 10;


Age,Gender,Education,total_co_users
18-24,Male,"Some college/university, no degree",144
18-24,Female,"Some college/university, no degree",41
25-34,Male,"Some college/university, no degree",33
18-24,Male,University degree,32
18-24,Male,Left school at 18,25
18-24,Female,University degree,24
25-34,Female,University degree,21
18-24,Male,Professional certificate/diploma,18
25-34,Female,Masters degree,17
25-34,Male,University degree,15


Vemos, mais uma vez, a predominância de homens jovens com ensino superior a concluir. Existe uma diferença de quase 400% entre homens e mulheres, na mesma faixa-etária e mesmo grau de escolaridade.

Em relação ao co-uso, fica bem claro a predominância de usuários de maconha (mais do que o dobro em relação às outras) e também o co-uso dessas drogas. Dos 417 usuários de cocaína, 374 também usam maconha. Dos 436 usuários de anfetamina, 398 também usam maconha. Ou seja, quase todos os usuários das outras duas drogas também usam maconha, o que deve estar relacionado ao fato dela ser a porta de entrada para o mundo das drogas.

A relação entre usuários de cocaína e anfetamina também é bem forte, sendo mais da metade de cada uma co-usuário da outra. Por fim, o co-uso triplo também teve um resultado expressivo, sendo metade dos usuários de cocaína e anfetamina.

Por fim, para responder a terceira pergunta, precisamos considerar tudo o que foi visto até então. Além disso, adicionarei mais dois parâmetros, indicando qual o perfil de usuários ocasionais e qual o perfil de usuários frequentes (viciados).

Lembrando, a terceira pergunta era:

3. Como o poder público poderia atuar para reduzir a quantidade de usuários?




In [0]:
%sql
SELECT Age, Gender, Education,
  SUM(CASE WHEN Cannabis = 'Used in last year' THEN 1 END) AS uso_ocasional_cannabis,
  SUM(CASE WHEN Coke = 'Used in last year' THEN 1 END) AS uso_ocasional_coke,
  SUM(CASE WHEN Amphet = 'Used in last year' THEN 1 END) AS uso_ocasional_amphet
FROM mvp.gold.v_drug_usage_fact
GROUP BY Age, Gender, Education
ORDER BY 
  uso_ocasional_cannabis DESC,
  uso_ocasional_coke DESC,
  uso_ocasional_amphet DESC
LIMIT 10;


Age,Gender,Education,uso_ocasional_cannabis,uso_ocasional_coke,uso_ocasional_amphet
18-24,Male,"Some college/university, no degree",27,55,63
25-34,Female,University degree,16,11,6
18-24,Female,"Some college/university, no degree",13,20,16
25-34,Female,Masters degree,13,7,5
18-24,Female,University degree,10,11,8
18-24,Male,University degree,9,10,10
35-44,Female,University degree,9,3,1
25-34,Male,University degree,7,9,3
35-44,Male,Professional certificate/diploma,7,6,3
25-34,Male,"Some college/university, no degree",6,19,14


In [0]:
%sql
SELECT Age, Gender, Education,
  SUM(CASE WHEN Cannabis = 'Used in last day' THEN 1 END) AS uso_frequente_cannabis,
  SUM(CASE WHEN Coke = 'Used in last day' THEN 1 END) AS uso_frequente_coke,
  SUM(CASE WHEN Amphet = 'Used in last day' THEN 1 END) AS uso_frequente_amphet
FROM mvp.gold.v_drug_usage_fact
GROUP BY Age, Gender, Education
ORDER BY 
  uso_frequente_cannabis DESC,
  uso_frequente_coke DESC,
  uso_frequente_amphet DESC
LIMIT 10;


Age,Gender,Education,uso_frequente_cannabis,uso_frequente_coke,uso_frequente_amphet
18-24,Male,"Some college/university, no degree",121,5,17
18-24,Female,"Some college/university, no degree",35,1,9
18-24,Male,Left school at 18,25,2,6
25-34,Male,"Some college/university, no degree",22,1,6
18-24,Male,University degree,22,1,4
18-24,Female,University degree,15,2,null
18-24,Male,Professional certificate/diploma,13,null,3
25-34,Female,University degree,12,1,7
25-34,Male,University degree,11,1,7
25-34,Male,Professional certificate/diploma,10,1,3


Olhando o resultado de uso ocasional, vemos o perfil predominante, já definido anteriormente, em primeiro lugar mas com baixo número de usuários de maconha. Aqui, as outras drogas acabam superando.
Também vemos uma presença maior de jovens mulheres com nível superior completo, superando até os homens na faixa-etária de 25-34 anos.

Porém, quando olhamos para o resultado de uso frequente, o cenário já conhecido retorna, com maior presença de homens e alto número de usuários de maconha, confirmando que o uso da maconha tende a se tornar vício enquanto que o uso de cocaína e anfetamina tendem a se relacionar com ambientes de festas e acontecimentos pontuais.

Com isso, fica evidente a necessidade de atuação do poder público entre os jovens nos ambientes de faculdade/universidade. Porém, acredito que tenha uma importância ainda maior as campanhas de prevenção enquanto os jovens ainda não chegaram nesse mundo de liberdade e curtição, sendo necessária a presença em escolas e colégios.


CONCLUSÃO

AUTO-AVALIAÇÃO